In [ ]:
import roveranalyzer as r 
import geopandas as g
import pandas as pd
import contextily  as ctx
from shapely.geometry import Polygon, box, Point
from matplotlib import pyplot as plt
import numpy as np
import math

In [ ]:
def create_grid(minx, miny, maxx, maxy, x_size=10, y_size=10):
    """
    Create a regular grid with given size in the given bound
    """
    num_x = math.floor((maxx - minx) / x_size)
    num_y = math.floor((maxy - miny) / y_size)
    cells = []
    _id = 0
    for x in range(num_x):
        x_min = minx + x_size * x 
        for y in range(num_y):
            y_min = miny + y_size * y 
            cells.append([_id, 'none', '#99382C',  box(x_min, y_min, x_min + x_size, y_min + y_size)])
            _id += 1
    
    return cells


def rnd_point_in_cell(bound, size, x, y):
    

In [ ]:
t = g.GeoDataFrame(
    {"name": ["bound", "used_area", "freiheit"],
     "geometry": [
         box( 11.512107,48.117927,  11.624853, 48.186458),
         box( 11.5567247,48.1462727 , 11.6140008,48.1702838),
         box( 11.58529, 48.16144, 11.58702, 48.16239)
     ]
    },
    crs="EPSG:4326"
)
# EPSG:4326             # https://epsg.io/4326 WGS84 World Geodetic System [degree]
t = t.to_crs(epsg=3857)  # https://epsg.io/3857 WGS84 / Pseudo-Mercator [meter] (Goolge, OpenStreetMap, ...)
# gt.to_crs(epsg=32632)   # https://epsg.io/32632 UTM zone 32N [meter]
t = t.drop(labels=[0, 1])

# CRS offset to work with smaller numbers around Müncher Freiheit. 
offset = np.array([1.289e+06, 6.133e+06])
c_muc_f = np.array([780, 845]) + offset # center point of crossing at Müncher Freiheit in EPSG:3857 (cartesian)
# bounding box to work with
bound = [c_muc_f[0] - 50, c_muc_f[1] - 50, c_muc_f[0]+150, c_muc_f[1] + 100]

# create regular grid inside bound
grid_df =  create_grid(*bound)
grid_df = g.GeoDataFrame(grid_df, columns=["name", "facecolor", "edgecolor", "geometry"], crs="EPSG:3857")

# create pedestrians
peds = []
peds.append([-1, '#187EAA', '#187EAA', Point([875, 888] + offset)])
peds.append([-1, '#187EAA', '#187EAA', Point([877, 889] + offset)])
peds.append([-1, '#187EAA', '#187EAA', Point([876, 888] + offset)])
peds.append([-1, '#187EAA', '#187EAA', Point([886, 887] + offset)])
peds.append([-1, '#187EAA', '#187EAA', Point([889, 889] + offset)])
peds.append([-1, '#187EAA', '#187EAA', Point([892, 890] + offset)])
peds.append([-1, '#187EAA', '#187EAA', Point([875, 908] + offset)])
peds.append([-1, '#187EAA', '#187EAA', Point([845, 890] + offset)])
peds = g.GeoDataFrame(peds, columns = ["name", "facecolor", "edgecolor", "geometry"], crs="EPSG:3857")


[Spatial Joins](https://geopandas.org/en/stable/gallery/spatial_joins.html?highlight=read_file)
https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html
https://contextily.readthedocs.io/en/latest/intro_guide.html#Using-transparent-layers


In [ ]:
ax = grid_df.plot(figsize=(16, 10), alpha=1.0,  edgecolor=grid_df.edgecolor, facecolor=grid_df.facecolor)
ctx.add_basemap(ax, zoom=18)
peds.plot(ax=ax, edgecolor=peds.edgecolor, facecolor=peds.facecolor, markersize=5)
ax.set_xlabel("Easting [meters]")
ax.set_ylabel("Northing [meters]")
ax.set_title("WSG 84 Pseudo-Mercator EPSG:3857")

In [ ]:
m = [11.36077, 48.0616244, 11.7229099, 48.2481162]
bound = box(692000.49, 5337200.59, 692323.49, 5337525.59)
cell = box(692313.49, 5337515.59, 692323.49, 5337525.59)
d = {"name": ["bound", "cell1"], 'geometry': [bound, cell]}
df_utm = g.GeoDataFrame(d, crs="EPSG:32632")
df = df_utm.to_crs(epsg=3857)

In [ ]:
ax = df.plot(figsize=(10, 10), alpha=0.5, facecolor='none', edgecolor='r')
ctx.add_basemap(ax, zoom = 18)
# m = ctx.Place("Munich")
# m.plot()
# df.crs